In [3]:
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from sklearn.linear_model import LinearRegression
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from lightgbm import LGBMRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
import pandas as pd
import numpy as np
import skforecast
import lightgbm
from xgboost import XGBRegressor
import xgboost
from numpy.random import Generator, PCG64

%load_ext pyinstrument

The pyinstrument extension is already loaded. To reload it, use:
  %reload_ext pyinstrument


In [4]:
print(skforecast.__version__)
print(lightgbm.__version__)
print(xgboost.__version__)

0.7.0
3.3.5
1.7.4


In [5]:
rng = Generator(PCG64())
rng.standard_normal()
n=10000
y = pd.Series(
    data=rng.standard_normal(size=n),
    index=pd.date_range(start='2000-01-01', periods=n, freq='D')
)

forecaster = ForecasterAutoreg(
    regressor=LGBMRegressor(n_estimators=500, learning_rate=0.01, max_depth=5),
    lags=50
)

In [7]:
%%pyinstrument
forecaster.fit(y=y)

_     ._   __/__   _ _  _  _ _/_   Recorded: 20:05:39  Samples:  331
 /_//_/// /_\ / //_// / //_'/ //     Duration: 0.601     CPU time: 4.094
/   _/                      v4.4.0

Program: c:\anaconda\envs\skforecast_p10_7\lib\site-packages\ipykernel_launcher.py --ip=127.0.0.1 --stdin=9013 --control=9006 --hb=9005 --Session.signature_scheme="hmac-sha256" --Session.key=b"32656c63-2f02-4c14-a9e0-02a832ecf71b" --shell=9007 --transport="tcp" --iopub=9014 --f="c:\Users\Joaquín Amat\AppData\Roaming\jupyter\runtime\kernel-v2-14940SjtlzqLdsDo7.json"

0.599 <module>  ..\..\..\..\Local\Temp\ipykernel_13052\1827134211.py:1
`- 0.599 ForecasterAutoreg.fit  skforecast\ForecasterAutoreg\ForecasterAutoreg.py:434
      [93 frames hidden]  skforecast, lightgbm, <built-in>, pan...
         0.474 Booster.update  lightgbm\basic.py:2964

In [10]:
%%pyinstrument
_ = forecaster.predict(steps=100)

_     ._   __/__   _ _  _  _ _/_   Recorded: 20:06:13  Samples:  26
 /_//_/// /_\ / //_// / //_'/ //     Duration: 0.027     CPU time: 0.062
/   _/                      v4.4.0

Program: c:\anaconda\envs\skforecast_p10_7\lib\site-packages\ipykernel_launcher.py --ip=127.0.0.1 --stdin=9013 --control=9006 --hb=9005 --Session.signature_scheme="hmac-sha256" --Session.key=b"32656c63-2f02-4c14-a9e0-02a832ecf71b" --shell=9007 --transport="tcp" --iopub=9014 --f="c:\Users\Joaquín Amat\AppData\Roaming\jupyter\runtime\kernel-v2-14940SjtlzqLdsDo7.json"

0.027 <module>  ..\..\..\..\Local\Temp\ipykernel_13052\360425224.py:1
`- 0.027 ForecasterAutoreg.predict  skforecast\ForecasterAutoreg\ForecasterAutoreg.py:561
      [36 frames hidden]  skforecast, lightgbm, <built-in>, skl...
         0.015 inner_predict  lightgbm\basic.py:896

In [14]:
%%pyinstrument

_ = backtesting_forecaster(
    forecaster=forecaster,
    y=y,
    initial_train_size=500,
    steps=1,
    metric='mean_absolute_error'
)

_     ._   __/__   _ _  _  _ _/_   Recorded: 20:08:48  Samples:  15124
 /_//_/// /_\ / //_// / //_'/ //     Duration: 15.733    CPU time: 102.234
/   _/                      v4.4.0

Program: c:\anaconda\envs\skforecast_p10_7\lib\site-packages\ipykernel_launcher.py --ip=127.0.0.1 --stdin=9013 --control=9006 --hb=9005 --Session.signature_scheme="hmac-sha256" --Session.key=b"32656c63-2f02-4c14-a9e0-02a832ecf71b" --shell=9007 --transport="tcp" --iopub=9014 --f="c:\Users\Joaquín Amat\AppData\Roaming\jupyter\runtime\kernel-v2-14940SjtlzqLdsDo7.json"

15.732 <module>  ..\..\..\..\Local\Temp\ipykernel_13052\2636043759.py:1
`- 15.732 backtesting_forecaster  skforecast\model_selection\model_selection.py:622
      [1790 frames hidden]  skforecast, lightgbm, numpy, ctypes, ...

In [15]:
%%pyinstrument

_ = backtesting_forecaster(
    forecaster=forecaster,
    y=y,
    initial_train_size=500,
    steps=10,
    metric='mean_absolute_error',
    refit=True
)

c:\anaconda\envs\skforecast_p10_7\lib\site-packages\skforecast\model_selection\model_selection.py:370: RuntimeWarning: The forecaster will be fit 950 times. This can take substantial amounts of time. If not feasible, try with `refit = False`. 

  warnings.warn(


_     ._   __/__   _ _  _  _ _/_   Recorded: 20:10:01  Samples:  185823
 /_//_/// /_\ / //_// / //_'/ //     Duration: 246.724   CPU time: 1765.281
/   _/                      v4.4.0

Program: c:\anaconda\envs\skforecast_p10_7\lib\site-packages\ipykernel_launcher.py --ip=127.0.0.1 --stdin=9013 --control=9006 --hb=9005 --Session.signature_scheme="hmac-sha256" --Session.key=b"32656c63-2f02-4c14-a9e0-02a832ecf71b" --shell=9007 --transport="tcp" --iopub=9014 --f="c:\Users\Joaquín Amat\AppData\Roaming\jupyter\runtime\kernel-v2-14940SjtlzqLdsDo7.json"

246.723 <module>  ..\..\..\..\Local\Temp\ipykernel_13052\3641819500.py:1
`- 246.723 backtesting_forecaster  skforecast\model_selection\model_selection.py:622
      [4190 frames hidden]  skforecast, lightgbm, <built-in>, cty...
         193.372 Booster.update  lightgbm\basic.py:2964

In [16]:
%%pyinstrument

# Grid search de hiperparámetros
# ==============================================================================
forecaster = ForecasterAutoregDirect(
                regressor = LGBMRegressor(max_depth=4),
                steps     = 36,
                lags      = 24 # Este valor será remplazado en el grid search
             )

# Lags utilizados como predictores
lags_grid = [[1, 2, 3, 23, 24], [1, 2, 3, 23, 24, 25, 47, 48, 49]]

# Hiperparámetros del regresor
param_grid = {'n_estimators': [100, 500],
              'max_depth': [4, 6]}

resultados_grid = grid_search_forecaster(
                        forecaster         = forecaster,
                        y                  = y,
                        param_grid         = param_grid,
                        lags_grid          = lags_grid,
                        steps              = 36,
                        metric             = 'mean_absolute_error',
                        refit              = False,
                        initial_train_size = 500,
                        return_best        = True,
                        verbose            = False
                  )

Number of models compared: 8.


loop lags_grid: 100%|███████████████████████████████████████| 2/2 [00:49<00:00, 24.55s/it]


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3 23 24] 
  Parameters: {'max_depth': 4, 'n_estimators': 100}
  Backtesting metric: 0.870228819012527



_     ._   __/__   _ _  _  _ _/_   Recorded: 20:31:13  Samples:  43720
 /_//_/// /_\ / //_// / //_'/ //     Duration: 51.837    CPU time: 348.031
/   _/                      v4.4.0

Program: c:\anaconda\envs\skforecast_p10_7\lib\site-packages\ipykernel_launcher.py --ip=127.0.0.1 --stdin=9013 --control=9006 --hb=9005 --Session.signature_scheme="hmac-sha256" --Session.key=b"32656c63-2f02-4c14-a9e0-02a832ecf71b" --shell=9007 --transport="tcp" --iopub=9014 --f="c:\Users\Joaquín Amat\AppData\Roaming\jupyter\runtime\kernel-v2-14940SjtlzqLdsDo7.json"

51.837 <module>  ..\..\..\..\Local\Temp\ipykernel_13052\3554055108.py:1
`- 51.826 grid_search_forecaster  skforecast\model_selection\model_selection.py:803
      [3554 frames hidden]  skforecast, lightgbm, numpy, ctypes, ...
         13.514 inner_predict  lightgbm\basic.py:896
         15.615 Booster.update  lightgbm\basic.py:2964

In [17]:
np.random.seed(123)
n = 10_000
data = pd.Series(data = np.random.normal(size=n))

class DummyRegressor(LinearRegression):
    """
    Dummy regressor with dummy fit and predict methods.
    """
    
    def fit(self, X, y):
        pass

    def predict(self, y):
        predictions = np.ones(shape = len(y))
        return predictions


In [18]:
%%pyinstrument
forecaster = ForecasterAutoreg(
                 regressor = DummyRegressor(),
                 lags      = 24
             )

forecaster.fit(y=data)

_     ._   __/__   _ _  _  _ _/_   Recorded: 20:32:27  Samples:  9
 /_//_/// /_\ / //_// / //_'/ //     Duration: 0.010     CPU time: 0.016
/   _/                      v4.4.0

Program: c:\anaconda\envs\skforecast_p10_7\lib\site-packages\ipykernel_launcher.py --ip=127.0.0.1 --stdin=9013 --control=9006 --hb=9005 --Session.signature_scheme="hmac-sha256" --Session.key=b"32656c63-2f02-4c14-a9e0-02a832ecf71b" --shell=9007 --transport="tcp" --iopub=9014 --f="c:\Users\Joaquín Amat\AppData\Roaming\jupyter\runtime\kernel-v2-14940SjtlzqLdsDo7.json"

0.009 <module>  ..\..\..\..\Local\Temp\ipykernel_13052\3697894503.py:1
|- 0.005 Booster.__del__  lightgbm\basic.py:2656
|     [2 frames hidden]  lightgbm
|- 0.003 ForecasterAutoreg.fit  skforecast\ForecasterAutoreg\ForecasterAutoreg.py:434
|     [19 frames hidden]  skforecast, pandas
`- 0.001 [self]  None

In [19]:
%%pyinstrument

forecaster = ForecasterAutoreg(
                 regressor = HistGradientBoostingRegressor(max_iter=10, random_state=123),
                 lags      = 24
             )

forecaster.fit(y=data)

_     ._   __/__   _ _  _  _ _/_   Recorded: 20:32:27  Samples:  151
 /_//_/// /_\ / //_// / //_'/ //     Duration: 0.171     CPU time: 0.516
/   _/                      v4.4.0

Program: c:\anaconda\envs\skforecast_p10_7\lib\site-packages\ipykernel_launcher.py --ip=127.0.0.1 --stdin=9013 --control=9006 --hb=9005 --Session.signature_scheme="hmac-sha256" --Session.key=b"32656c63-2f02-4c14-a9e0-02a832ecf71b" --shell=9007 --transport="tcp" --iopub=9014 --f="c:\Users\Joaquín Amat\AppData\Roaming\jupyter\runtime\kernel-v2-14940SjtlzqLdsDo7.json"

0.171 <module>  ..\..\..\..\Local\Temp\ipykernel_13052\1359942394.py:1
`- 0.171 ForecasterAutoreg.fit  skforecast\ForecasterAutoreg\ForecasterAutoreg.py:434
      [194 frames hidden]  skforecast, sklearn, <built-in>, nump...

In [20]:
%%pyinstrument
forecaster = ForecasterAutoreg(
                 regressor = DummyRegressor(),
                 lags      = 24
             )

forecaster.fit(y=data)

_     ._   __/__   _ _  _  _ _/_   Recorded: 20:32:28  Samples:  2
 /_//_/// /_\ / //_// / //_'/ //     Duration: 0.003     CPU time: 0.000
/   _/                      v4.4.0

Program: c:\anaconda\envs\skforecast_p10_7\lib\site-packages\ipykernel_launcher.py --ip=127.0.0.1 --stdin=9013 --control=9006 --hb=9005 --Session.signature_scheme="hmac-sha256" --Session.key=b"32656c63-2f02-4c14-a9e0-02a832ecf71b" --shell=9007 --transport="tcp" --iopub=9014 --f="c:\Users\Joaquín Amat\AppData\Roaming\jupyter\runtime\kernel-v2-14940SjtlzqLdsDo7.json"

0.002 <module>  ..\..\..\..\Local\Temp\ipykernel_13052\3697894503.py:1
`- 0.002 ForecasterAutoreg.fit  skforecast\ForecasterAutoreg\ForecasterAutoreg.py:434
      [5 frames hidden]  skforecast

In [21]:
%%pyinstrument

_ = forecaster.predict(steps=1000)

_     ._   __/__   _ _  _  _ _/_   Recorded: 20:32:29  Samples:  19
 /_//_/// /_\ / //_// / //_'/ //     Duration: 0.019     CPU time: 0.031
/   _/                      v4.4.0

Program: c:\anaconda\envs\skforecast_p10_7\lib\site-packages\ipykernel_launcher.py --ip=127.0.0.1 --stdin=9013 --control=9006 --hb=9005 --Session.signature_scheme="hmac-sha256" --Session.key=b"32656c63-2f02-4c14-a9e0-02a832ecf71b" --shell=9007 --transport="tcp" --iopub=9014 --f="c:\Users\Joaquín Amat\AppData\Roaming\jupyter\runtime\kernel-v2-14940SjtlzqLdsDo7.json"

0.019 <module>  ..\..\..\..\Local\Temp\ipykernel_13052\3639615597.py:1
`- 0.019 ForecasterAutoreg.predict  skforecast\ForecasterAutoreg\ForecasterAutoreg.py:561
      [22 frames hidden]  skforecast, <__array_function__ inter...
         0.019 ForecasterAutoreg._recursive_predict  skforecast\ForecasterAutoreg\ForecasterAutoreg.py:511
         |- 0.004 DummyRegressor.predict  ..\..\..\..\Local\Temp\ipykernel_13052\3445934768.py:13
         |  `- 0.004 ones  numpy\core\numeric.py:149
         |        [6 frames hidden]  numpy, <__array_function__ internals>...

In [22]:
data = pd.read_csv("data.csv")
data.date_time = pd.to_datetime(data.date_time)
data= data.set_index('date_time')
data = data.asfreq('H')

# Split train-validation-test
# ==============================================================================
end_train = '2012-03-31 23:59:00'
end_validation = '2012-08-31 23:59:00'
data_train = data.loc[: end_train, :]
data_val   = data.loc[end_train:end_validation, :]
data_test  = data.loc[end_validation:, :]

print(f"Dates train      : {data_train.index.min()} --- {data_train.index.max()}  (n={len(data_train)})")
print(f"Dates validacion : {data_val.index.min()} --- {data_val.index.max()}  (n={len(data_val)})")
print(f"Dates test       : {data_test.index.min()} --- {data_test.index.max()}  (n={len(data_test)})")

# Create forecaster
# ==============================================================================
forecaster = ForecasterAutoreg(
                 regressor = XGBRegressor(random_state=123),
                 lags = 24
             )

Dates train      : 2011-01-01 00:00:00 --- 2012-03-31 23:00:00  (n=10944)
Dates validacion : 2012-04-01 00:00:00 --- 2012-08-31 23:00:00  (n=3672)
Dates test       : 2012-09-01 00:00:00 --- 2012-12-31 23:00:00  (n=2928)


In [23]:
%%pyinstrument
forecaster.fit(y= data.loc[:end_validation, 'users'],)

_     ._   __/__   _ _  _  _ _/_   Recorded: 20:32:52  Samples:  118
 /_//_/// /_\ / //_// / //_'/ //     Duration: 0.646     CPU time: 4.281
/   _/                      v4.4.0

Program: c:\anaconda\envs\skforecast_p10_7\lib\site-packages\ipykernel_launcher.py --ip=127.0.0.1 --stdin=9013 --control=9006 --hb=9005 --Session.signature_scheme="hmac-sha256" --Session.key=b"32656c63-2f02-4c14-a9e0-02a832ecf71b" --shell=9007 --transport="tcp" --iopub=9014 --f="c:\Users\Joaquín Amat\AppData\Roaming\jupyter\runtime\kernel-v2-14940SjtlzqLdsDo7.json"

0.644 <module>  ..\..\..\..\Local\Temp\ipykernel_13052\2000120308.py:1
`- 0.643 ForecasterAutoreg.fit  skforecast\ForecasterAutoreg\ForecasterAutoreg.py:434
      [110 frames hidden]  skforecast, xgboost, copy, pandas, <b...

In [24]:
%%pyinstrument
_ = backtesting_forecaster(
                   forecaster         = forecaster,
                   y                  = data.loc[:end_validation, 'users'], # Train and validation data
                   steps              = 36,
                   refit              = False,
                   metric             = 'mean_squared_error',
                   initial_train_size = len(data_train)
)

_     ._   __/__   _ _  _  _ _/_   Recorded: 20:32:54  Samples:  4247
 /_//_/// /_\ / //_// / //_'/ //     Duration: 4.733     CPU time: 33.156
/   _/                      v4.4.0

Program: c:\anaconda\envs\skforecast_p10_7\lib\site-packages\ipykernel_launcher.py --ip=127.0.0.1 --stdin=9013 --control=9006 --hb=9005 --Session.signature_scheme="hmac-sha256" --Session.key=b"32656c63-2f02-4c14-a9e0-02a832ecf71b" --shell=9007 --transport="tcp" --iopub=9014 --f="c:\Users\Joaquín Amat\AppData\Roaming\jupyter\runtime\kernel-v2-14940SjtlzqLdsDo7.json"

4.732 <module>  ..\..\..\..\Local\Temp\ipykernel_13052\136797954.py:1
`- 4.731 backtesting_forecaster  skforecast\model_selection\model_selection.py:622
      [935 frames hidden]  skforecast, xgboost, sklearn, inspect...

In [25]:
%%pyinstrument
# Grid search of hyperparameters and lags
# ==============================================================================
# Regressor hyperparameters
param_grid = {
    'n_estimators': [100],
    'max_depth': [3, 5, 10],
    'learning_rate': [0.01, 0.1]
}

# Lags used as predictors
lags_grid = [48, 72]

results_grid = grid_search_forecaster(
                   forecaster         = forecaster,
                   y                  = data.loc[:end_validation, 'users'], # Train and validation data
                   param_grid         = param_grid,
                   lags_grid          = lags_grid,
                   steps              = 36,
                   refit              = False,
                   metric             = 'mean_squared_error',
                   initial_train_size = len(data_train), # Model is trained with training data
                   fixed_train_size   = False,
                   return_best        = True,
                   verbose            = False
               )

Number of models compared: 12.


loop lags_grid: 100%|███████████████████████████████████████| 2/2 [01:33<00:00, 46.67s/it]


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72] 
  Parameters: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}
  Backtesting metric: 14407.853484255475



_     ._   __/__   _ _  _  _ _/_   Recorded: 20:33:06  Samples:  62952
 /_//_/// /_\ / //_// / //_'/ //     Duration: 96.006    CPU time: 616.328
/   _/                      v4.4.0

Program: c:\anaconda\envs\skforecast_p10_7\lib\site-packages\ipykernel_launcher.py --ip=127.0.0.1 --stdin=9013 --control=9006 --hb=9005 --Session.signature_scheme="hmac-sha256" --Session.key=b"32656c63-2f02-4c14-a9e0-02a832ecf71b" --shell=9007 --transport="tcp" --iopub=9014 --f="c:\Users\Joaquín Amat\AppData\Roaming\jupyter\runtime\kernel-v2-14940SjtlzqLdsDo7.json"

96.006 <module>  ..\..\..\..\Local\Temp\ipykernel_13052\244285910.py:1
`- 96.005 grid_search_forecaster  skforecast\model_selection\model_selection.py:803
      [2898 frames hidden]  skforecast, xgboost, sklearn, inspect...
         30.375 Booster.update  xgboost\core.py:1897

In [26]:
%%pyinstrument
# Grid search of hyperparameters and lags
# ==============================================================================
# Regressor hyperparameters
param_grid = {
    'n_estimators': [1000],
    'max_depth': [3, 5, 10],
    'learning_rate': [0.01, 0.1]
}

# Lags used as predictors
lags_grid = [48, 72]

results_grid = grid_search_forecaster(
                   forecaster         = forecaster,
                   y                  = data.loc[:end_validation, 'users'], # Train and validation data
                   param_grid         = param_grid,
                   lags_grid          = lags_grid,
                   steps              = 36,
                   refit              = False,
                   metric             = 'mean_squared_error',
                   initial_train_size = len(data_train), # Model is trained with training data
                   fixed_train_size   = False,
                   return_best        = True,
                   verbose            = False
               )

Number of models compared: 12.


loop lags_grid: 100%|██████████████████████████████████████| 2/2 [05:00<00:00, 150.08s/it]


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72] 
  Parameters: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 1000}
  Backtesting metric: 13605.107261105946



_     ._   __/__   _ _  _  _ _/_   Recorded: 20:35:10  Samples:  87379
 /_//_/// /_\ / //_// / //_'/ //     Duration: 320.085   CPU time: 2241.844
/   _/                      v4.4.0

Program: c:\anaconda\envs\skforecast_p10_7\lib\site-packages\ipykernel_launcher.py --ip=127.0.0.1 --stdin=9013 --control=9006 --hb=9005 --Session.signature_scheme="hmac-sha256" --Session.key=b"32656c63-2f02-4c14-a9e0-02a832ecf71b" --shell=9007 --transport="tcp" --iopub=9014 --f="c:\Users\Joaquín Amat\AppData\Roaming\jupyter\runtime\kernel-v2-14940SjtlzqLdsDo7.json"

320.084 <module>  ..\..\..\..\Local\Temp\ipykernel_13052\558181007.py:1
`- 320.083 grid_search_forecaster  skforecast\model_selection\model_selection.py:803
      [2919 frames hidden]  skforecast, xgboost, <built-in>, typi...
         231.306 Booster.update  xgboost\core.py:1897

In [31]:
%%pyinstrument


forecaster = ForecasterAutoreg(
                 regressor = XGBRegressor(random_state=123, n_estimators=500),
                 lags = 24
             )

             
# Grid search of hyperparameters and lags
# ==============================================================================
# Regressor hyperparameters
param_grid = {
    'n_estimators': [100, 500],
    'max_depth': [3, 5, 10],
    'learning_rate': [0.01, 0.1]
}

# Lags used as predictors
lags_grid = [48, 72]

results_grid = grid_search_forecaster(
                   forecaster         = forecaster,
                   y                  = data.loc[:end_validation, 'users'], # Train and validation data
                   param_grid         = param_grid,
                   lags_grid          = lags_grid,
                   steps              = 36,
                   refit              = False,
                   metric             = 'mean_squared_error',
                   initial_train_size = len(data_train), # Model is trained with training data
                   fixed_train_size   = False,
                   return_best        = True,
                   verbose            = False
               )

Number of models compared: 24.


loop lags_grid:   0%|                                               | 0/2 [00:42<?, ?it/s]


KeyboardInterrupt: 

In [27]:
import session_info
session_info.show(html=False)

-----
lightgbm            3.3.5
numpy               1.23.5
pandas              1.5.3
pyinstrument        4.4.0
session_info        1.0.0
skforecast          0.7.0
sklearn             1.2.1
xgboost             1.7.4
-----
IPython             8.10.0
jupyter_client      8.0.3
jupyter_core        5.2.0
notebook            6.5.4
-----
Python 3.10.0 | packaged by conda-forge | (default, Nov 10 2021, 13:20:59) [MSC v.1916 64 bit (AMD64)]
Windows-10-10.0.19045-SP0
-----
Session information updated at 2023-05-22 21:06
